<a href="https://colab.research.google.com/github/DONGJUSEO/2020_Big-Contest/blob/master/%EC%B2%A0%EB%8F%84_%ED%83%88%EC%84%A0%EA%B3%84%EC%88%98_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델링
### 1) AutoML
### 2) ExtraTrees
### 3) TabNet
### 4) LSTM (Darts)

In [ ]:
## AutoML

'''
y_train.columns = ['YL_M1_B1_W1']
data = pd.concat([x_train, y_train], axis = 1)

reg_test_1 = setup(data = data,
                   target = 'YL_M1_B1_W1',
                   train_size = 0.90,
                   data_split_shuffle = False,
                   normalize = False,
                   #normalize_method = 'minmax',
                   #polynomial_features = True,
                   #polynomial_degree = 2,
                   #remove_multicollinearity = True,
                   #multicollinearity_threshol = 0.9,
                   #feature_selection = True,
                   #feature_selection_method = 'classic',
                   session_id = 20230806,
                   verbose = True)

# 모델 탐색
gc.collect()
start = time.time()
best = compare_models(sort = 'mape', fold = 3, n_select = 1, include = ['xgboost', 'rf'])
print("time :", time.time() - start)

# 모델 생성
automl_model = create_model(best)

# 하이퍼파라미터 튜닝
gc.collect()
start = time.time()
tuned_automl_model = tune_model(automl_model, optimize = 'MAPE', n_iter = 3)
print("time :", time.time() - start)

# 모델링
automl_pred = predict_model(tuned_automl_model, data = x_valid.reset_index(drop = True))
automl_pred = pd.DataFrame(automl_pred.loc[:, 'prediction_label'])
y_valid = y_valid.reset_index(drop = True)
y_valid.columns = ['prediction_label']

MAPE(y_valid, automl_pred)
'''

In [ ]:
## ExtraTrees

'''
#!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
from sklearn.ensemble import ExtraTreesRegressor
gc.collect()

# 하이퍼파라미터 튜닝
def et_bo(n_estimators, max_depth, min_samples_split, min_samples_leaf) :
    et_params = {'n_estimators' : int(round(n_estimators)),
                 'max_depth' : int(round(max_depth)),
                 'min_samples_split' : int(round(min_samples_split)),
                 'min_samples_leaf' : int(round(min_samples_leaf)),}
    et_model = ExtraTreesRegressor(**et_params)
    et_model.fit(x_train, y_train)
    et_pred = et_model.predict(x_valid)
    score = -mean_absolute_error(y_valid, et_pred) # maximize이므로 음수로 지정
    return score
et_parameter_bounds = {'n_estimators' : (200, 900),
                       'max_depth' : (3, 9),
                       'min_samples_split' : (2, 9),
                       'min_samples_leaf' : (1, 5)
                       ,}
bo_et = BayesianOptimization(f = et_bo, pbounds = et_parameter_bounds, random_state = 20230807)
bo_et.maximize(init_points = 2, n_iter = 10)
max_params = bo_et.max['params']
max_params['n_estimators'] = int(max_params['n_estimators'])
max_params['max_depth'] = int(max_params['max_depth'])
max_params['min_samples_split'] = int(max_params['min_samples_split'])
max_params['min_samples_leaf'] = int(max_params['min_samples_leaf'])
print(max_params)

# 모델링
et_model = ExtraTreesRegressor(n_estimators = max_params['n_estimators'],
                                max_depth = max_params['max_depth'],
                                min_samples_split = max_params['min_samples_split'],
                                min_samples_leaf = max_params['min_samples_leaf'],
                                random_state = 20230807)
et_model.fit(x_train, y_train)
et_pred = et_model.predict(x_valid)

def MAPE(y_test, y_pred) :
	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
MAPE(y_valid.reset_index(drop = True), et_pred)

# 예측 시각화
plt.figure(figsize = (16, 8))
plt.plot(y_valid.reset_index(drop = True), color = 'red', alpha = 0.75)
plt.plot(et_pred, color = 'green', alpha = 1)
plt.title('ExtraTrees', fontsize = 20)
plt.xlabel('Index', fontsize = 12)
plt.ylabel('Duration', fontsize = 12)
plt.legend(['Test Set', 'Prediction'])
plt.show()

# Feature Importance
importance = et_model.feature_importances_
feature = x_train.columns
et_f_i = pd.DataFrame()
et_f_i['feature'] = feature
et_f_i['importances'] = importance
et_f_i.sort_values('importances', ascending = False, inplace = True)
et_f_i.reset_index(drop = True, inplace = True)
sns.barplot(data = et_f_i, x = 'importances', y = 'feature')
plt.show()
'''

In [ ]:
## TabNet

'''
gc.collect()

# 필수 전처리 작업
clf_x_train = x_train.to_numpy()
clf_y_train = y_train.to_numpy().reshape(-1, 1)
clf_x_valid = x_valid.to_numpy()
clf_y_valid = y_valid.to_numpy().reshape(-1, 1)

# 하이퍼파라미터 튜닝
def clf_bo(n_d, gamma, momentum) :
    clf_params = {'n_d' : int(round(n_d)),
                  'gamma' : gamma,
                  'momentum' : momentum,}
    clf_model = TabNetRegressor(**clf_params,
                                optimizer_params = dict(lr = 0.01),
                                scheduler_params = {'step_size' : 10, 'gamma' : 0.9},
                                scheduler_fn = torch.optim.lr_scheduler.StepLR,
                                mask_type = 'entmax')
    clf_model.fit(clf_x_train, clf_y_train,
                  max_epochs = 200,
                  patience = 10,
                  batch_size = 128)
    clf_pred = clf_model.predict(clf_x_valid)
    score = -mean_absolute_error(clf_y_valid, clf_pred)
    return score
clf_parameter_bounds = {'n_d' : (8, 32),
                        'gamma' : (1, 2),
                        'momentum' : (0.01, 0.4),}
bo_clf = BayesianOptimization(f = clf_bo, pbounds = clf_parameter_bounds, random_state = 20230807)
bo_clf.maximize(init_points = 2, n_iter = 10)
max_params = bo_clf.max['params']
max_params['n_d'] = int(round(max_params['n_d']))
max_params['gamma'] = max_params['gamma']
max_params['momentum'] = max_params['momentum']
print(max_params)

# 모델링
clf_model = TabNetRegressor(n_d = max_params['n_d'],
                            n_a = max_params['n_d'],
                            gamma = max_params['gamma'],
                            momentum = max_params['momentum'],
                            optimizer_params = dict(lr = 0.01),
                            scheduler_params = {'step_size' : 10, 'gamma' : 0.9},
                            scheduler_fn = torch.optim.lr_scheduler.StepLR,
                            mask_type = 'entmax',
                            seed = 20230807)
clf_model.fit(clf_x_train, clf_y_train,
              max_epochs = 200,
              patience = 5,
              batch_size = 128)
clf_pred = clf_model.predict(clf_x_valid)
print(mean_absolute_percentage_error(clf_y_valid, clf_pred) * 100)
'''

# LSTM

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install darts
# import pandas as pd
# import numpy as np
# import time
# import gc
# import matplotlib.pyplot as plt
# import seaborn as sns
# import torch
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from darts import TimeSeries
# from darts.models import RNNModel, XGBModel
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from sklearn.metrics import mean_absolute_error
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# pd.options.display.float_format = '{:.5f}'.format

In [ ]:
# lane_data_s = pd.read_csv('/content/lane_data_s.csv')
# lane_data_c = pd.read_csv('/content/lane_data_c.csv')

# data_s30 = pd.read_csv('/content/data_s30.csv')
# data_s40 = pd.read_csv('/content/data_s40.csv')
# data_s50 = pd.read_csv('/content/data_s50.csv')
# data_s70 = pd.read_csv('/content/data_s70.csv')
# data_s100 = pd.read_csv('/content/data_s100.csv')

# data_c30 = pd.read_csv('/content/data_c30.csv')
# data_c40 = pd.read_csv('/content/data_c40.csv')
# data_c50 = pd.read_csv('/content/data_c50.csv')
# data_c70 = pd.read_csv('/content/data_c70.csv')
# data_c100 = pd.read_csv('/content/data_c100.csv')

In [ ]:
# # 주성분 분석
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 2)
# printcipalComponents = pca.fit_transform(x)
# principalDf = pd.DataFrame(data = printcipalComponents, columns = ['principal component1', 'principal component2'])
# pca.explained_variance_ratio_
# sum(pca.explained_variance_ratio_)

In [ ]:
'''
def modelling(data, lane_data, target, length, x) :
    # 0 ~ 10000 : 학습데이터, 10001 ~ 11999 : 검증데이터
    li = ['Distance', 'YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']
    li.remove(target)
    m_data = pd.merge(data, lane_data, on = 'Distance')
    m_data_train = pd.merge(data, lane_data, on = 'Distance')[0 : 10001]
    m_data_train = m_data_train.drop(li, axis = 1)
    m_data_test = pd.merge(data, lane_data, on = 'Distance')[10001 : ]
    m_data_test = m_data_test.drop(li, axis = 1)

    # # 이상치 제거
    # q1, q3 = m_data_train[target].quantile(q = 0.25), m_data_train[target].quantile(q = 0.75)
    # iqr = q3 - q1
    # minimum, maximum = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    # m_data_train = m_data_train[(minimum <= m_data_train[target]) & (m_data_train[target] <= maximum)].reset_index(drop = True)

    # # 변수 선택
    # corr = m_data_train.corr(method = 'pearson')
    # index = corr[(corr[target] >= 0.2) | (corr[target] <= -0.2)].index.tolist()
    # m_data_train = m_data_train.loc[:, index]
    # m_data_test = m_data_test.loc[:, index]

    # 변수 선택
    columns = m_data_train.columns.tolist()
    columns_li_1 = []
    for i in columns :
        if (x) in i :
            columns_li_1.append(i)
    m_data_train = m_data_train.loc[:, columns_li_1]
    m_data_test = m_data_test.loc[:, columns_li_1]

    # 표준화
    y_train = m_data_train[target]
    x_train = m_data_train.drop([target], axis = 1)
    x_test = m_data_test.drop([target], axis = 1)
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = pd.DataFrame(scaler.transform(x_train))
    x_test = pd.DataFrame(scaler.transform(x_test))
    scaled_df = pd.concat([x_train, x_test], axis = 0).reset_index(drop = True)

    # Time 생성 후 병합
    time = pd.DataFrame(pd.date_range('2023-08-13', periods = len(scaled_df), freq = 's'))
    time.columns = ['time']
    x_train = pd.concat([time.iloc[0 : len(x_train), :], x_train], axis = 1)
    y_train = pd.concat([time.iloc[0 : len(y_train), :], y_train], axis = 1)
    x_test = pd.concat([time.iloc[len(x_train) - length : , :].reset_index(drop = True), scaled_df.iloc[len(x_train) - length : , :].reset_index(drop = True)], axis = 1)

    # 필수 전처리 : 시계열 데이터로 변환
    x_train = TimeSeries.from_dataframe(x_train, time_col= "time")
    y_train = TimeSeries.from_dataframe(y_train, time_col= "time")
    x_test = TimeSeries.from_dataframe(x_test, time_col= "time")

    # 모델링
    gc.collect()
    my_stopper = EarlyStopping(monitor = "train_loss",
                               patience = 5,
    )
    pl_trainer_kwargs = {"callbacks" : [my_stopper]}

    model = RNNModel(input_chunk_length = length,
                     output_chunk_length = 1,
                     model = 'LSTM',
                     dropout = 0,
                     batch_size = 32,
                     n_epochs = 100,
                     optimizer_cls = torch.optim.Adam,
                     optimizer_kwargs = {'lr' : 0.001},
                     random_state = 20230813,
                     pl_trainer_kwargs = pl_trainer_kwargs)
    model.fit(series = y_train, future_covariates = x_train)
    prediction = model.predict(n = len(x_test) - length, future_covariates = x_test, batch_size = 32)

    return prediction
'''

In [ ]:
# s30_1 = modelling(data_s30, lane_data_s, 'YL_M1_B1_W1', 40, 'W1')
# s30_2 = modelling(data_s30, lane_data_s, 'YR_M1_B1_W1', 40, 'W1')
# s30_3 = modelling(data_s30, lane_data_s, 'YL_M1_B1_W2', 40, 'W2')
# s30_4 = modelling(data_s30, lane_data_s, 'YR_M1_B1_W2', 40, 'W2')
# s30_1 = pd.DataFrame(TimeSeries.values(s30_1), columns = ['YL_M1_B1_W1_s30'])
# s30_2 = pd.DataFrame(TimeSeries.values(s30_2), columns = ['YR_M1_B1_W1_s30'])
# s30_3 = pd.DataFrame(TimeSeries.values(s30_3), columns = ['YL_M1_B1_W2_s30'])
# s30_4 = pd.DataFrame(TimeSeries.values(s30_4), columns = ['YR_M1_B1_W2_s30'])
# final_data = pd.concat([s30_1, s30_2, s30_3, s30_4], axis = 1)
# final_data.to_csv('/content/predict_data_s30.csv')

# s40_1 = modelling(data_s40, lane_data_s, 'YL_M1_B1_W1', 40, 'W1')
# s40_2 = modelling(data_s40, lane_data_s, 'YR_M1_B1_W1', 40, 'W1')
# s40_3 = modelling(data_s40, lane_data_s, 'YL_M1_B1_W2', 40, 'W2')
# s40_4 = modelling(data_s40, lane_data_s, 'YR_M1_B1_W2', 40, 'W2')
# s40_1 = pd.DataFrame(TimeSeries.values(s40_1), columns = ['YL_M1_B1_W1_s40'])
# s40_2 = pd.DataFrame(TimeSeries.values(s40_2), columns = ['YR_M1_B1_W1_s40'])
# s40_3 = pd.DataFrame(TimeSeries.values(s40_3), columns = ['YL_M1_B1_W2_s40'])
# s40_4 = pd.DataFrame(TimeSeries.values(s40_4), columns = ['YR_M1_B1_W2_s40'])
# final_data = pd.concat([s40_1, s40_2, s40_3, s40_4], axis = 1)
# final_data.to_csv('/content/predict_data_s40.csv')

# s50_1 = modelling(data_s50, lane_data_s, 'YL_M1_B1_W1', 40, 'W1')
# s50_2 = modelling(data_s50, lane_data_s, 'YR_M1_B1_W1', 40, 'W1')
# s50_3 = modelling(data_s50, lane_data_s, 'YL_M1_B1_W2', 40, 'W2')
# s50_4 = modelling(data_s50, lane_data_s, 'YR_M1_B1_W2', 40, 'W2')
# s50_1 = pd.DataFrame(TimeSeries.values(s50_1), columns = ['YL_M1_B1_W1_s50'])
# s50_2 = pd.DataFrame(TimeSeries.values(s50_2), columns = ['YR_M1_B1_W1_s50'])
# s50_3 = pd.DataFrame(TimeSeries.values(s50_3), columns = ['YL_M1_B1_W2_s50'])
# s50_4 = pd.DataFrame(TimeSeries.values(s50_4), columns = ['YR_M1_B1_W2_s50'])
# final_data = pd.concat([s50_1, s50_2, s50_3, s50_4], axis = 1)
# final_data.to_csv('/content/predict_data_s50.csv')

# s70_1 = modelling(data_s70, lane_data_s, 'YL_M1_B1_W1', 40, 'W1')
# s70_2 = modelling(data_s70, lane_data_s, 'YR_M1_B1_W1', 40, 'W1')
# s70_3 = modelling(data_s70, lane_data_s, 'YL_M1_B1_W2', 40, 'W2')
# s70_4 = modelling(data_s70, lane_data_s, 'YR_M1_B1_W2', 40, 'W2')
# s70_1 = pd.DataFrame(TimeSeries.values(s70_1), columns = ['YL_M1_B1_W1_s70'])
# s70_2 = pd.DataFrame(TimeSeries.values(s70_2), columns = ['YR_M1_B1_W1_s70'])
# s70_3 = pd.DataFrame(TimeSeries.values(s70_3), columns = ['YL_M1_B1_W2_s70'])
# s70_4 = pd.DataFrame(TimeSeries.values(s70_4), columns = ['YR_M1_B1_W2_s70'])
# final_data = pd.concat([s70_1, s70_2, s70_3, s70_4], axis = 1)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_data_s70.csv')

# s100_1 = modelling(data_s100, lane_data_s, 'YL_M1_B1_W1', 40, 'W1')
# s100_2 = modelling(data_s100, lane_data_s, 'YR_M1_B1_W1', 40, 'W1')
# s100_3 = modelling(data_s100, lane_data_s, 'YL_M1_B1_W2', 40, 'W2')
# s100_4 = modelling(data_s100, lane_data_s, 'YR_M1_B1_W2', 40, 'W2')
# s100_1 = pd.DataFrame(TimeSeries.values(s100_1), columns = ['YL_M1_B1_W1_s100'])
# s100_2 = pd.DataFrame(TimeSeries.values(s100_2), columns = ['YR_M1_B1_W1_s100'])
# s100_3 = pd.DataFrame(TimeSeries.values(s100_3), columns = ['YL_M1_B1_W2_s100'])
# s100_4 = pd.DataFrame(TimeSeries.values(s100_4), columns = ['YR_M1_B1_W2_s100'])
# final_data = pd.concat([s100_1, s100_2, s100_3, s100_4], axis = 1)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_data_s100.csv')

# c30_1 = modelling(data_c30, lane_data_c, 'YL_M1_B1_W1', 40, 'W1')
# c30_2 = modelling(data_c30, lane_data_c, 'YR_M1_B1_W1', 40, 'W1')
# c30_3 = modelling(data_c30, lane_data_c, 'YL_M1_B1_W2', 40, 'W2')
# c30_4 = modelling(data_c30, lane_data_c, 'YR_M1_B1_W2', 40, 'W2')
# c30_1 = pd.DataFrame(TimeSeries.values(c30_1), columns = ['YL_M1_B1_W1_c30'])
# c30_2 = pd.DataFrame(TimeSeries.values(c30_2), columns = ['YR_M1_B1_W1_c30'])
# c30_3 = pd.DataFrame(TimeSeries.values(c30_3), columns = ['YL_M1_B1_W2_c30'])
# c30_4 = pd.DataFrame(TimeSeries.values(c30_4), columns = ['YR_M1_B1_W2_c30'])
# final_data = pd.concat([c30_1, c30_2, c30_3, c30_4], axis = 1)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_data_c30.csv')

# c40_1 = modelling(data_c40, lane_data_c, 'YL_M1_B1_W1', 40, 'W1')
# c40_2 = modelling(data_c40, lane_data_c, 'YR_M1_B1_W1', 40, 'W1')
# c40_3 = modelling(data_c40, lane_data_c, 'YL_M1_B1_W2', 40, 'W2')
# c40_4 = modelling(data_c40, lane_data_c, 'YR_M1_B1_W2', 40, 'W2')
# c40_1 = pd.DataFrame(TimeSeries.values(c40_1), columns = ['YL_M1_B1_W1_c40'])
# c40_2 = pd.DataFrame(TimeSeries.values(c40_2), columns = ['YR_M1_B1_W1_c40'])
# c40_3 = pd.DataFrame(TimeSeries.values(c40_3), columns = ['YL_M1_B1_W2_c40'])
# c40_4 = pd.DataFrame(TimeSeries.values(c40_4), columns = ['YR_M1_B1_W2_c40'])
# final_data = pd.concat([c40_1, c40_2, c40_3, c40_4], axis = 1)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_data_c40.csv')

# c50_1 = modelling(data_c50, lane_data_c, 'YL_M1_B1_W1', 40, 'W1')
# c50_2 = modelling(data_c50, lane_data_c, 'YR_M1_B1_W1', 40, 'W1')
# c50_3 = modelling(data_c50, lane_data_c, 'YL_M1_B1_W2', 40, 'W2')
# c50_4 = modelling(data_c50, lane_data_c, 'YR_M1_B1_W2', 40, 'W2')
# c50_1 = pd.DataFrame(TimeSeries.values(c50_1), columns = ['YL_M1_B1_W1_c50'])
# c50_2 = pd.DataFrame(TimeSeries.values(c50_2), columns = ['YR_M1_B1_W1_c50'])
# c50_3 = pd.DataFrame(TimeSeries.values(c50_3), columns = ['YL_M1_B1_W2_c50'])
# c50_4 = pd.DataFrame(TimeSeries.values(c50_4), columns = ['YR_M1_B1_W2_c50'])
# final_data = pd.concat([c50_1, c50_2, c50_3, c50_4], axis = 1)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_data_c50.csv')

# c70_1 = modelling(data_c70, lane_data_c, 'YL_M1_B1_W1', 40, 'W1')
# c70_2 = modelling(data_c70, lane_data_c, 'YR_M1_B1_W1', 40, 'W1')
# c70_3 = modelling(data_c70, lane_data_c, 'YL_M1_B1_W2', 40, 'W2')
# c70_4 = modelling(data_c70, lane_data_c, 'YR_M1_B1_W2', 40, 'W2')
# c70_1 = pd.DataFrame(TimeSeries.values(c70_1), columns = ['YL_M1_B1_W1_c70'])
# c70_2 = pd.DataFrame(TimeSeries.values(c70_2), columns = ['YR_M1_B1_W1_c70'])
# c70_3 = pd.DataFrame(TimeSeries.values(c70_3), columns = ['YL_M1_B1_W2_c70'])
# c70_4 = pd.DataFrame(TimeSeries.values(c70_4), columns = ['YR_M1_B1_W2_c70'])
# final_data = pd.concat([c70_1, c70_2, c70_3, c70_4], axis = 1)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_data_c70.csv')

# c100_1 = modelling(data_c100, lane_data_c, 'YL_M1_B1_W1', 40, 'W1')
# c100_2 = modelling(data_c100, lane_data_c, 'YR_M1_B1_W1', 40, 'W1')
# c100_3 = modelling(data_c100, lane_data_c, 'YL_M1_B1_W2', 40, 'W2')
# c100_4 = modelling(data_c100, lane_data_c, 'YR_M1_B1_W2', 40, 'W2')
# c100_1 = pd.DataFrame(TimeSeries.values(c100_1), columns = ['YL_M1_B1_W1_c100'])
# c100_2 = pd.DataFrame(TimeSeries.values(c100_2), columns = ['YR_M1_B1_W1_c100'])
# c100_3 = pd.DataFrame(TimeSeries.values(c100_3), columns = ['YL_M1_B1_W2_c100'])
# c100_4 = pd.DataFrame(TimeSeries.values(c100_4), columns = ['YR_M1_B1_W2_c100'])
# final_data = pd.concat([c100_1, c100_2, c100_3, c100_4], axis = 1)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_data_c100.csv')

# 최종

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import gc
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
lane_data_s = pd.read_csv('/content/lane_data_s.csv')
lane_data_c = pd.read_csv('/content/lane_data_c.csv')

data_s30 = pd.read_csv('/content/data_s30.csv')
data_s40 = pd.read_csv('/content/data_s40.csv')
data_s50 = pd.read_csv('/content/data_s50.csv')
data_s70 = pd.read_csv('/content/data_s70.csv')
data_s100 = pd.read_csv('/content/data_s100.csv')

data_c30 = pd.read_csv('/content/data_c30.csv')
data_c40 = pd.read_csv('/content/data_c40.csv')
data_c50 = pd.read_csv('/content/data_c50.csv')
data_c70 = pd.read_csv('/content/data_c70.csv')
data_c100 = pd.read_csv('/content/data_c100.csv')

In [ ]:
data_s30 = pd.read_csv('/content/data_s30.csv')
data_s30 = data_s30[0 : 10001]
lane_data_s = pd.read_csv('/content/lane_data_s.csv')

# 이상치 제거
target = 'YL_M1_B1_W1'
q1, q3 = data_s30[target].quantile(q = 0.25), data_s30[target].quantile(q = 0.75)
iqr = q3 - q1
minimum, maximum = q1 - 3 * iqr, q3 + 3 * iqr
data_s30 = data_s30[(minimum <= data_s30[target]) & (data_s30[target] <= maximum)].reset_index(drop = True)
distance = data_s30.loc[:, 'Distance']

# 변수 선택
columns_index = data_s30.columns.tolist()
columns_li_1 = []
for i in columns_index :
    if ('W1') in i :
        columns_li_1.append(i)
columns_li_2 = []
for j in columns_li :
    if ('L') in j :
        columns_li_2.append(j)

data_s30 = data_s30.loc[:, columns_li_2]
data_s30 = pd.concat([distance, data_s30], axis = 1)
data_s30 = pd.merge(data_s30, lane_data_s, on = 'Distance')
data_s30 = data_s30.drop(['V_M1_B1_W1_L'], axis = 1)

# 다중공선성
# data_s30.corr(method = 'pearson')
# data_s30.corr(method ='spearman')
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(data_s30.values, i) for i in range(data_s30.shape[1])]
# vif["features"] = data_s30.columns
# vif = vif.sort_values(by = "VIF Factor", ascending = True)
# vif = vif.reset_index().drop(columns = 'index')
# vif

# Length 설정 (4, 6)
target = data_s30[target]
for i in range(0, 40) :
    print('Length %d : ' % i, stats.pearsonr(target[0 : len(target) - (1 + i)], target[1 + i : ]))

In [ ]:
def modelling(data, lane_data, target, length) :
    train_data = data[0 : 10001]
    m_data = pd.merge(data, lane_data, on = 'Distance')
    test_data = m_data[10001 : ]

    # 이상치 제거
    q1, q3 = train_data[target].quantile(q = 0.25), train_data[target].quantile(q = 0.75)
    iqr = q3 - q1
    minimum, maximum = q1 - 3 * iqr, q3 + 3 * iqr
    train_data  = train_data [(minimum <= train_data [target]) & (train_data [target] <= maximum)].reset_index(drop = True)
    distance = train_data .loc[:, 'Distance']

    # 변수 선택
    if target == 'YL_M1_B1_W1' :
        x, y, z = 'W1', 'L', 'V_M1_B1_W1_L'
    elif target == 'YR_M1_B1_W1' :
        x, y, z = 'W1', 'R', 'V_M1_B1_W1_R'
    elif target == 'YL_M1_B1_W2' :
        x, y, z = 'W2', 'L', 'V_M1_B1_W2_L'
    else :
        x, y, z = 'W2', 'R', 'V_M1_B1_W2_R'

    columns_index = train_data.columns.tolist()
    columns_li_1 = []
    for i in columns_index :
        if (x) in i :
            columns_li_1.append(i)
    columns_li_2 = []
    for j in columns_li_1 :
        if (y) in j :
            columns_li_2.append(j)

    train_data = train_data.loc[:, columns_li_2]
    train_data = pd.concat([distance, train_data], axis = 1)
    train_data = pd.merge(train_data, lane_data, on = 'Distance')
    train_data = train_data.drop(['Distance', z], axis = 1)
    train_target = pd.DataFrame(train_data[target])
    train_data = train_data.drop(target, axis = 1)

    # 스케일링
    test_data = test_data.loc[:, train_data.columns.tolist()]
    scaler = StandardScaler()
    scaler.fit(train_data)
    scaled_train_data = pd.DataFrame(scaler.transform(train_data))
    scaled_data = pd.concat([scaled_train_data, train_target], axis = 1)
    scaled_test_data = pd.DataFrame(scaler.transform(test_data))

    # 모형 구축
    def build_model(train_data, target_data, p) :
        x = pd.DataFrame()
        y = pd.DataFrame()
        for i in range(0, len(train_data) - p) :
            x_empty = np.array([])
            for j in range(0, p) :
                empty = np.array(train_data.iloc[i + j, :])
                x_empty = np.concatenate((x_empty, empty), axis = 0)
            x_empty = pd.DataFrame(x_empty).T
            x = x.append(x_empty)
            y_empty = pd.DataFrame(np.array(target_data.iloc[i + p, :]))
            y = y.append(y_empty.T)
        x = x.reset_index(drop = True)
        y = y.reset_index(drop = True)
        return x, y
    train_data, target_data = build_model(scaled_data, train_target, length)

    # 모델링
    from sklearn.multioutput import MultiOutputRegressor
    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(train_data, target_data)

    # 롤링
    scaled_df = pd.concat([scaled_data, scaled_test_data], axis = 0).reset_index(drop = True)
    for i in range(0, len(scaled_test_data)) :
        part_scaled_df = pd.concat([pd.DataFrame(scaled_df.loc[len(scaled_data) - length + i, :]).T.reset_index(drop = True),
                                    pd.DataFrame(scaled_df.loc[len(scaled_data) - length + i + 1, :]).T.reset_index(drop = True),
                                    pd.DataFrame(scaled_df.loc[len(scaled_data) - length + i + 2, :]).T.reset_index(drop = True),
                                    pd.DataFrame(scaled_df.loc[len(scaled_data) - length + i + 3, :]).T.reset_index(drop = True),], axis = 1)
        part_scaled_df.columns = range(0, len(part_scaled_df.columns), 1)
        xgb_pred = xgb_model.predict(part_scaled_df)
        scaled_df.loc[len(scaled_data) + i, [target]] = xgb_pred.tolist()
    scaled_df = scaled_df.loc[len(scaled_data) : , [target]].reset_index(drop = True)

    return scaled_df

data_1 = modelling(data_s30, lane_data_s, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_s30, lane_data_s, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_s30, lane_data_s, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_s30, lane_data_s, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/1.csv')

In [ ]:
# data_1 = modelling(data_s30, lane_data_s, 'YL_M1_B1_W1', 4)
# data_2 = modelling(data_s30, lane_data_s, 'YR_M1_B1_W1', 4)
# data_3 = modelling(data_s30, lane_data_s, 'YL_M1_B1_W2', 4)
# data_4 = modelling(data_s30, lane_data_s, 'YR_M1_B1_W2', 4)
# final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/1.csv')

data_1 = modelling(data_s40, lane_data_s, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_s40, lane_data_s, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_s40, lane_data_s, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_s40, lane_data_s, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/2.csv')

data_1 = modelling(data_s50, lane_data_s, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_s50, lane_data_s, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_s50, lane_data_s, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_s50, lane_data_s, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/3.csv')

data_1 = modelling(data_s70, lane_data_s, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_s70, lane_data_s, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_s70, lane_data_s, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_s70, lane_data_s, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/4.csv')

data_1 = modelling(data_s100, lane_data_s, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_s100, lane_data_s, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_s100, lane_data_s, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_s100, lane_data_s, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/5.csv')

data_1 = modelling(data_c30, lane_data_c, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_c30, lane_data_c, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_c30, lane_data_c, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_c30, lane_data_c, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/6.csv')

data_1 = modelling(data_c40, lane_data_c, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_c40, lane_data_c, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_c40, lane_data_c, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_c40, lane_data_c, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/7.csv')

data_1 = modelling(data_c50, lane_data_c, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_c50, lane_data_c, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_c50, lane_data_c, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_c50, lane_data_c, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/8.csv')

data_1 = modelling(data_c70, lane_data_c, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_c70, lane_data_c, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_c70, lane_data_c, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_c70, lane_data_c, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/9.csv')

data_1 = modelling(data_c100, lane_data_c, 'YL_M1_B1_W1', 4)
data_2 = modelling(data_c100, lane_data_c, 'YR_M1_B1_W1', 4)
data_3 = modelling(data_c100, lane_data_c, 'YL_M1_B1_W2', 4)
data_4 = modelling(data_c100, lane_data_c, 'YR_M1_B1_W2', 4)
final_data = pd.concat([data_1, data_2, data_3, data_4], axis = 1)
final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/10.csv')

In [ ]:
def modelling(data, lane_data, length) :
    m_data = pd.merge(data, lane_data, on = 'Distance')
    m_data = m_data.drop('Distance', axis = 1)
    m_data_train = m_data[0 : 10001]
    m_data_test = m_data[10001 : ]

    target = ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']
    m_data_x_train = m_data_train.drop(target, axis = 1)
    m_data_y_train = m_data_train.loc[:, target]
    m_data_x_test = m_data_test.drop(target, axis = 1)

    # # 스케일링
    # scaler = MinMaxScaler()
    # scaler.fit(m_data_x_train)
    # scaled_data = pd.DataFrame(scaler.transform(m_data_x_train))
    # scaled_data_x_train = pd.concat([scaled_data, m_data_y_train], axis = 1)
    # scaled_data_x_test = pd.DataFrame(scaler.transform(m_data_x_test))

    # 스케일링
    m_data_x = pd.concat([m_data_x_train, m_data_x_test], axis = 0)
    scaler = StandardScaler()
    scaler.fit(m_data_x)
    scaled_data = pd.DataFrame(scaler.transform(m_data_x))
    scaled_data_x_train = pd.concat([scaled_data[0 : 10001], m_data_y_train], axis = 1)
    scaled_data_x_test = scaled_data[10001 : ]

    # 모형 구축
    def build_model(train_data, target_data, p) :
        x = pd.DataFrame()
        y = pd.DataFrame()
        for i in range(0, len(train_data) - p) :
            x_empty = np.array([])
            for j in range(0, p) :
                empty = np.array(train_data.iloc[i + j, :])
                x_empty = np.concatenate((x_empty, empty), axis = 0)
            x_empty = pd.DataFrame(x_empty).T
            x = x.append(x_empty)
            y_empty = pd.DataFrame(np.array(target_data.iloc[i + p, :]))
            y = y.append(y_empty.T)
        x = x.reset_index(drop = True)
        y = y.reset_index(drop = True)
        return x, y
    train_data, target_data = build_model(scaled_data_x_train, m_data_y_train, length)

    # 모델링
    from sklearn.multioutput import MultiOutputRegressor
    xgb_multi_model = MultiOutputRegressor(xgb.XGBRegressor())
    xgb_multi_model.fit(train_data, target_data)

    # 롤링
    scaled_df = pd.concat([scaled_data_x_train, scaled_data_x_test], axis = 0).reset_index(drop = True)
    for i in range(0, len(m_data_test)) :
        part_scaled_df = pd.concat([pd.DataFrame(scaled_df.loc[10001 - length + i, :]).T.reset_index(drop = True),
                                    pd.DataFrame(scaled_df.loc[10001 - length + i + 1, :]).T.reset_index(drop = True),
                                    pd.DataFrame(scaled_df.loc[10001 - length + i + 2, :]).T.reset_index(drop = True)], axis = 1)
        part_scaled_df.columns = range(0, len(part_scaled_df.columns), 1)

        pred = xgb_multi_model.predict(part_scaled_df)
        scaled_df.loc[10001 + i, ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']] = pred.tolist()

    return scaled_df

In [ ]:
# result_1 = modelling(data_s30, lane_data_s, 3)
# result_1 = result_1.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
# result_1.to_csv('/content/drive/MyDrive/Colab Notebooks/1.csv')

# result_2 = modelling(data_s40, lane_data_s, 3)
# result_2 = result_2.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
# result_2.to_csv('/content/drive/MyDrive/Colab Notebooks/2.csv')

# result_3 = modelling(data_s50, lane_data_s, 3)
# result_3 = result_3.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
# result_3.to_csv('/content/drive/MyDrive/Colab Notebooks/3.csv')

# result_4 = modelling(data_s70, lane_data_s, 3)
# result_4 = result_4.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
# result_4.to_csv('/content/drive/MyDrive/Colab Notebooks/4.csv')

# result_5 = modelling(data_s100, lane_data_s, 3)
# result_5 = result_5.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
# result_5.to_csv('/content/drive/MyDrive/Colab Notebooks/5.csv')

result_6 = modelling(data_c30, lane_data_c, 3)
result_6 = result_6.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
result_6.to_csv('/content/drive/MyDrive/Colab Notebooks/6.csv')

result_7 = modelling(data_c40, lane_data_c, 3)
result_7 = result_7.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
result_7.to_csv('/content/drive/MyDrive/Colab Notebooks/7.csv')

result_8 = modelling(data_c50, lane_data_c, 3)
result_8 = result_8.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
result_8.to_csv('/content/drive/MyDrive/Colab Notebooks/8.csv')

result_9 = modelling(data_c70, lane_data_c, 3)
result_9 = result_9.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
result_9.to_csv('/content/drive/MyDrive/Colab Notebooks/9.csv')

result_10 = modelling(data_c100, lane_data_c, 3)
result_10 = result_10.loc[10001 : , ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']].reset_index(drop = True)
result_10.to_csv('/content/drive/MyDrive/Colab Notebooks/10.csv')

In [ ]:
# def MAPE(y_test, y_pred) :
# 	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
# print('1 : ', MAPE(y_valid.iloc[:, 0].values, pd.DataFrame(pred).iloc[:, 0].values))
# print('2 : ', MAPE(y_valid.iloc[:, 1].values, pd.DataFrame(pred).iloc[:, 1].values))
# print('3 : ', MAPE(y_valid.iloc[:, 2].values, pd.DataFrame(pred).iloc[:, 2].values))
# print('4 : ', MAPE(y_valid.iloc[:, 3].values, pd.DataFrame(pred).iloc[:, 3].values))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
lane_data_s = pd.read_csv('/content/lane_data_s.csv')
lane_data_c = pd.read_csv('/content/lane_data_c.csv')

data_s30 = pd.read_csv('/content/data_s30.csv')
data_s40 = pd.read_csv('/content/data_s40.csv')
data_s50 = pd.read_csv('/content/data_s50.csv')
data_s70 = pd.read_csv('/content/data_s70.csv')
data_s100 = pd.read_csv('/content/data_s100.csv')

data_c30 = pd.read_csv('/content/data_c30.csv')
data_c40 = pd.read_csv('/content/data_c40.csv')
data_c50 = pd.read_csv('/content/data_c50.csv')
data_c70 = pd.read_csv('/content/data_c70.csv')
data_c100 = pd.read_csv('/content/data_c100.csv')

In [ ]:
import statsmodels.api as sm

data = data_s30
lane_data = lane_data_s

m_data = pd.merge(data, lane_data, on = 'Distance')
m_data = m_data.drop('Distance', axis = 1)
m_data_train = m_data[0 : 10001]
m_data_test = m_data[10001 : ]

# 이상치 제거
target = 'YL_M1_B1_W1'
q1, q3 = m_data_train[target].quantile(q = 0.25), m_data_train[target].quantile(q = 0.75)
iqr = q3 - q1
minimum, maximum = q1 - 3 * iqr, q3 + 3 * iqr
m_data_train = m_data_train[(minimum <= m_data_train[target]) & (m_data_train[target] <= maximum)].reset_index(drop = True)

# 스케일링
scaler = MinMaxScaler()
scaler.fit(m_data_train)
scaled_m_data_train = scaler.transform(m_data_train)
columns = m_data_train.columns
scaled_m_data_train = pd.DataFrame(scaled_m_data_train, columns = columns)

# 학습모형 구축
target = ['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']
scaled_m_data_x_train = scaled_m_data_train.drop(target[1 : ], axis = 1)
scaled_m_data_y_train = scaled_m_data_train[target[0]]
scaled_m_data_x_train = scaled_m_data_x_train[0 : -1]
scaled_m_data_y_train = list(scaled_m_data_y_train[1 : ])

# 상수항 추가
scaled_m_data_x_train = sm.add_constant(scaled_m_data_x_train, has_constant = 'add')

# 데이터 분할
x_train, x_valid, y_train, y_valid = train_test_split(scaled_m_data_x_train,
                                                      scaled_m_data_y_train,
                                                      test_size = 0.05,
                                                      shuffle = False,
                                                      random_state = 1)
# OLS 검정
multi_model = sm.OLS(y_train, x_train)
fitted_multi_model = multi_model.fit()
fitted_multi_model.summary()

In [ ]:
pred = fitted_multi_model.predict(x_valid)

In [ ]:
def MAPE(y_test, y_pred) :
	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
MAPE(y_valid, pred.reset_index(drop = True))

In [ ]:
y_valid
pred.reset_index(drop = True)

In [ ]:
plt.plot(y_valid)
plt.plot(pred.reset_index(drop = True))